In [ ]:
import tensorflow as tf

In [ ]:
tf.__version__

'2.15.0'

In [4]:
# Nối gg collab vào gg drive -> để lưu dữ liệu vào trong thư mục của gg drive
from google.colab import drive
drive.mount("/content/drive") # nối collab hiện tại với thư mục hiện hành

MessageError: Error: credential propagation was unsuccessful

In [ ]:
cd ..


In [ ]:
cd /content/drive/MyDrive/ML Test

In [ ]:
link = "https://d17h27t6h515a5.cloudfront.net/topher/2017/February/5898cd6f_traffic-signs-data/traffic-signs-data.zip"

In [ ]:
# load data từ đường link
!pip install wget

In [ ]:
import wget

wget.download(link)

In [ ]:
# lưu địa chỉ giải nén
data = "./" # giải nén ra chính thư mục hiện tại

!unzip -q traffic-signs-data.zip -d $data

In [ ]:
train_link = data + "train.p"
valid_link = data + "valid.p"
test_link = data + "test.p"

In [ ]:
# do các đường link đang dạng .p nên ta chưa thể đọc nó ngay
import pickle

In [ ]:
with open(train_link, mode = "rb") as f:
  train = pickle.load(f)

In [ ]:
with open(valid_link, mode = "rb") as f:
  valid = pickle.load(f)
with open(test_link, mode = "rb") as f:
  test = pickle.load(f)

In [ ]:
train

là 1 kiểu dictionary

In [ ]:
trainX = train["features"]
trainY = train["labels"]

In [ ]:
validX = valid["features"]
validY = valid["labels"]

testX = test["features"]
testY = test['labels']

In [ ]:
trainX.shape

có 34799 bức ảnh 32x32x3 -> 1 bức ảnh 3 lớp và chiều mỗi lớp 32x32

In [ ]:
trainX[0].shape

In [ ]:
!pip install matplotlib

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
plt.imshow(trainX[0])

In [ ]:
plt.imshow(trainX[50])

Bộ data đẹp do được tăng cường vì được chụp trong nhiều điều kiện khác nhau

In [ ]:
trainY[1], trainY[50]

In [ ]:
classNames = {0: 'Speed limit (20km/h)',
 1: 'Speed limit (30km/h)',
 2: 'Speed limit (50km/h)',
 3: 'Speed limit (60km/h)',
 4: 'Speed limit (70km/h)',
 5: 'Speed limit (80km/h)',
 6: 'End of speed limit (80km/h)',
 7: 'Speed limit (100km/h)',
 8: 'Speed limit (120km/h)',
 9: 'No passing',
 10: 'No passing for vehicles over 3.5 metric tons',
 11: 'Right-of-way at the next intersection',
 12: 'Priority road',
 13: 'Yield',
 14: 'Stop',
 15: 'No vehicles',
 16: 'Vehicles over 3.5 metric tons prohibited',
 17: 'No entry',
 18: 'General caution',
 19: 'Dangerous curve to the left',
 20: 'Dangerous curve to the right',
 21: 'Double curve',
 22: 'Bumpy road',
 23: 'Slippery road',
 24: 'Road narrows on the right',
 25: 'Road work',
 26: 'Traffic signals',
 27: 'Pedestrians',
 28: 'Children crossing',
 29: 'Bicycles crossing',
 30: 'Beware of ice/snow',
 31: 'Wild animals crossing',
 32: 'End of all speed and passing limits',
 33: 'Turn right ahead',
 34: 'Turn left ahead',
 35: 'Ahead only',
 36: 'Go straight or right',
 37: 'Go straight or left',
 38: 'Keep right',
 39: 'Keep left',
 40: 'Roundabout mandatory',
 41: 'End of no passing',
 42: 'End of no passing by vehicles over 3.5 metric tons'}



In [ ]:
classNames[trainY[1]]

NHẬN XÉT: Các ảnh có label giống nhau đặt liền kề nhau. Khi train model từng phần 1 mà mỗi phần có ảnh giống y hệt nhau sẽ ảnh hưởng đến tính tổng quát của model

In [ ]:
from sklearn.utils import shuffle

In [ ]:
#tráo các ảnh ứng luôn với các label
trainX, trainY = shuffle(trainX, trainY)

In [ ]:
# chuẩn hóa data TRAIN về dạng (0,1)

trainX = trainX.astype("float") / 255.0
validX = validX.astype("float") / 255.0
testX = testX.astype("float") / 255.0

In [ ]:
trainX[0]

In [ ]:
#  Chuyển label về dạng onehot

from sklearn.preprocessing import LabelBinarizer

In [ ]:
lb = LabelBinarizer()

In [ ]:
trainY = lb.fit_transform(trainY)
validY = lb.fit_transform(validY)

In [ ]:
trainY[0]

# XÂY DỰNG MODEL

In [ ]:
from tensorflow.keras.models import Model
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import BatchNormalization
from tensorflow.keras.layers import AveragePooling2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import Activation
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Flatten
from tensorflow.keras.layers import Input
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import concatenate
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.optimizers import SGD

In [ ]:
model = Sequential()

In [ ]:
# định nghĩa đầu vào

width = 32
height = 32
classes = 43

In [ ]:
shape = (width, height, 3)

## TRÍCH XUẤT DẶC TRƯNG CỦA ẢNH

Convolution layer được thực hiện như 1 phép để xử lí ảnh đầu vào. Nó sẽ tạo ra kernel.


Kernel là ma trận có kích thước nhỏ so với ma trận đầu vào có nhiệm vụ trượt trên ma trận ảnh gốc. Filter này chứa các trọng số, và khi di chuyển qua ma trận đầu vào, nó thực hiện phép nhân và tổng hợp để tạo ra một đặc trưng mới ( có thể hiểu nôm na tạo luôn ra 1 ảnh mới với các features mới nhưng mình có thể kiểm soát được ) -> mục tiêu là để làm nổi bật lên các đặc trưng của ảnh ( ví dụ như trích xuất ra các hàng ngang của ảnh hay chiều dọc của ảnh )


Tuy nhiên trong deeplearning , các hệ số của kernel để trống để trong quá trình tìm loss func thì nó sẽ học và tự điền sao cho giá trị loss nhà nhỏ nhât

SỐ LƯỢNG KERNEL CHÍNH BẰNG SỐ LƯỢNG CHANNEL ĐẦU RA

In [ ]:
# SỬ DỤNG TÍCH CHẬP LẤY RA CÁC GIÁ TRỊ ĐẶC TRƯNG
model.add(Conv2D(32, (3,3), padding = "same", input_shape = shape))
# sử dụng 32 kernel 3x3
# padding = same nhằm mục đích giữ lại chiều của ảnh đầu vào là 32 x 32. Nếu ko có thì width và height của ảnh sẽ giảm , ảnh sẽ bị mất đi các cạnh do trong quá trình quét

In [ ]:
model.summary()

đầu ra của lớp này sẽ là 32x32, None sẽ thể hiện số lượng ảnh training. tổng param là 896 -> số lượng phép cần phải học

In [ ]:
# thêm lớp phi tuyến (hàm relu)

model.add(Activation("relu"))

In [ ]:
model.summary()

lớp Activation chỉ là hàm biến đổi, ko cần phải học gì cả nên số lượng param bằng 0

In [ ]:
# thêm vào lớp batchnorm
model.add(BatchNormalization())

hàm chuẩn hóa phân bố lại sao cho phân bố data đồng nhất lại như phân bố ban đầu vì sau 2 lớp trước phân bố data đã bị thay đổi rất mạnh -> chuẩn hóa lại sao cho độ thay đổi ko quá lớn


In [ ]:
model.summary()

In [ ]:
# thêm 1 layer conv
model.add(Conv2D(32, (3,3), padding = "same"))

#thêm 1 lớp activation
model.add(Activation("relu"))

# thêm vào lớp batchnorm
model.add(BatchNormalization())

In [ ]:
model.summary()

Vậy bao nhiêu lớp nữa là đủ?? cái này phụ thuộc vào data , thêm sao cho vừa đủ sao cho ko overfit cũng không underfit

MAXPOOLING cũng tạo ra kernel và trượt như conv tuy nhiên nó chỉ lấy phần tử lớn nhất trong 1 khoảng nó trượt qua ( mục đích là lấy ra các đặc trưng đã được làm nối bật qua hàm conv2d ở trên)

VIỆC NÀY SẼ LÀM THU NHỎ LẠI PHÉP ĐẦU RA


In [ ]:
 model.add(MaxPooling2D(pool_size=(2,2)))
 #kernel 2x2

In [ ]:
model.summary()

do chỉ có tác dụng trích xuất ra các đặc trưng để thu nhỏ đầu ra nên ko cần phải học

In [ ]:
model.add(Conv2D(64, (3,3), padding = "same"))
model.add(Activation("relu"))
model.add(BatchNormalization())
model.add(Conv2D(64, (3,3), padding = "same"))
model.add(Activation("relu"))
model.add(BatchNormalization())

In [ ]:
model.summary()

In [ ]:
 model.add(MaxPooling2D(pool_size=(2,2)))

In [ ]:
model.summary()

## PHÂN LOẠI ẢNH

Đầu vào của mạng này bắt buộc phải là 1 vector chứ không thể là ma trận

In [ ]:
model.add(Flatten())
# Duỗi ảnh ra thành 1 vector

In [ ]:
model.summary()

In [ ]:
# chuyển từ vector 4096,1 thành vector 512,1
model.add(Dense(512))
model.add(Activation("relu")) # bắt buộc trước hàm dense phải có hàm activation


In [ ]:
model.add(BatchNormalization())

In [ ]:
model.summary()

In [ ]:
# chuyển từ vector 521,1 thành vector classes,1
model.add(Dense(classes))


In [ ]:
# lớp cuối cùng dể phân loại biết các nào label nào sẽ là hàm softmax
model.add(Activation("softmax")) # bắt buộc trước hàm dense phải có hàm activation

In [ ]:
model.summary()

## Tăng cường data

In [ ]:
aug = ImageDataGenerator(rotation_range= 0.18, zoom_range= 0.15, width_shift_range= 0.2, height_shift_range=0.2, horizontal_flip=True)
# ảnh xoay 18% và zoom lên 15%
# dịch sang chiều ngang và chiều dọc 20%
# ảnh còn có thể lật lại

In [ ]:
learning_rate = 0.01


In [ ]:
epochs = 10
#epochs -> step -> batchsize
batch_size = 64

In [ ]:
opt = SGD(learning_rate = learning_rate, momentum = 0.9)
#momentum quán tính để thoát local để tìm điểm cực tiểu global

In [ ]:
model.compile(optimizer = opt, loss = "categorical_crossentropy", metrics = ['accuracy'])

In [ ]:
H = model.fit_generator(aug.flow(trainX, trainY, batch_size= batch_size), validation_data = (validX,validY), steps_per_epoch = trainX.shape[0]//batch_size,epochs = epochs, verbose = 1)

In [ ]:
model.save('modelTrafficSign.h5')

In [ ]:
H

In [ ]:
model.fit_generator(aug.flow(trainX, trainY, batch_size= batch_size), validation_data = (validX,validY), steps_per_epoch = trainX.shape[0]//batch_size,epochs = 5, verbose = 1)

In [1]:
model.save('modelTrafficSign.h5')

NameError: name 'model' is not defined

In [3]:
#lấy model đã lưu
import tensorflow as tf
saved_model = tf.keras.models.load_model('modelTrafficSign.h5')

OSError: No file or directory found at modelTrafficSign.h5

In [ ]:
result = saved_model.predict(testX[1:2])

In [ ]:
import numpy as np
final = np.argmax(result)

In [ ]:
classNames[final]

In [ ]:
plt.imshow(test["features"][1])